# SMPL

SMPL is a classical body model for human body shape and pose estimation. We can get joints position and the vertices of mesh from SMPL's results.

## Preparation

Download the SMPL's checkpoint and unzip it to `data_inputs/body_models/smpl`.

```
.
├── SMPL_FEMALE.pkl
├── SMPL_MALE.pkl
└── SMPL_NEUTRAL.pkl
```

## References

- [SMPL Project Page](https://smpl.is.tue.mpg.de/)
- [SMPL-made-simple-FAQs](https://files.is.tue.mpg.de/black/talks/SMPL-made-simple-FAQs.pdf)

## Tutorials

### Environment Preparation

In [46]:
# Packages you may use very often.
import torch
import numpy as np
from smplx import SMPL
from pytorch3d import transforms

# Things you don't need to care about. They are just for driving the tutorials.
from lib.utils.path_manager import PathManager
from lib.viewer.wis3d_utils import HWis3D as Wis3D
from lib.skeleton import Skeleton_SMPL24

pm = PathManager()

### Load SMPL model

SMPL has different models weights for different genders. Make sure you use the correct model for your project.

Usually, we just use the neutral model if we can't access the gender information. 

Here, we will just use neutral model for simplicity. You can try the other genders if you want. You can just re-assign the `body_model` variable in the next cell and re-run the remaining cells.

In [11]:
body_models = {}
genders = ['neutral', 'female', 'male']  # case insensitive

for gender in genders:
    body_models[gender] = SMPL(
            model_path = pm.inputs / 'body_models' / 'smpl',
            gender     = gender,
        )

In [30]:
# Prepare some parameters for later inference.
B = 150
body_model : SMPL = body_models['neutral']  # use neutral for example

# Prepare mesh template for later visualization.
# Tips: mesh = vertices + faces, and the faces are the indices of vertices, which won't change across SMPL's outputs.
mesh_temp : np.ndarray = body_model.faces  # (13776, 3)

### SMPL Inference

In [13]:
# Inference.
smpl_out = body_model(
        betas         = torch.zeros(B, 10),     # shape coefficients
        global_orient = torch.zeros(B, 1, 3),   # axis-angle representation
        body_pose     = torch.zeros(B, 23, 3),  # axis-angle representation
        transl        = torch.zeros(B, 3),
    )

# Check output.
joints : torch.Tensor = smpl_out.joints    # (B, 45, 3)
verts  : torch.Tensor = smpl_out.vertices  # (B, 7890 3)
print(joints.shape, verts.shape)

torch.Size([150, 45, 3]) torch.Size([150, 6890, 3])


### SMPL Parameters

Here we will learn the main for SMPL parameters through several demos. The main parameters are:

1. betas
2. global_orient
3. body_pose
4. transl

#### 1. betas | $\beta \in \R^{||\beta||}$

Betas control the shape of the model. Usually we use the default 10 shape coefficients. It depends on the model you load.

You may see this(below) before, that means you are using a model with 10 shape coefficients.

> "WARNING: You are using a SMPL model, with only 10 shape coefficients." 

In [47]:
def learn_betas(
    selected_component : int = 0,
    lower_bound : int = -5,
    upper_bound : int = +5,
):
    fake_betas = torch.zeros(B, 10)
    fake_betas[:, selected_component] = torch.linspace(lower_bound, upper_bound, B)

    # Inference.
    smpl_out = body_model(
            betas         = fake_betas,             # shape coefficients
            global_orient = torch.zeros(B, 1, 3),   # axis-angle representation
            body_pose     = torch.zeros(B, 23, 3),  # axis-angle representation
            transl        = torch.zeros(B, 3),
        )

    # Check output.
    joints : torch.Tensor = smpl_out.joints    # (B, 45 3)
    verts  : torch.Tensor = smpl_out.vertices  # (B, 7890 3)

    def visualize_results():
        """ This part is to visualize the results. You are supposed to ignore this part. """
        shape_wis3d = Wis3D(
                pm.outputs / 'wis3d',
                'SMPL-parameters',
            )

        shape_wis3d.add_motion_verts(
                verts  = verts,
                name   = f'shape: betas[:, {selected_component}] from {lower_bound} to {upper_bound}',
                offset = 0,
            )
        shape_wis3d.add_motion_skel(
                joints = joints[:, :24],
                bones  = Skeleton_SMPL24.bones,
                colors = Skeleton_SMPL24.bone_colors,
                name   = f'shape: betas[:, {selected_component}] from {lower_bound} to {upper_bound}',
                offset = 0,
            )
    visualize_results()

We will visualize the effects of the changes on certain coefficient.

Here, `learn_betas(k)` means we will visualize the SMPL outputs when the k-th coefficient is changed from -5 to +5.

In [ ]:
learn_betas(0)
learn_betas(1)
learn_betas(2)

Now start the viewer.

You will see that, the skeleton will mis-align with the mesh when the coefficients are very "sharp".
Some of the coefficient control the height, the length of the limbs, etc.

In [ ]:
# Start the server. (Remember to terminate the cell before going on.)
!wis3d --vis_dir {pm.outputs / 'wis3d'} --host 0.0.0.0 --port 19090

#### 2. global_orient | $\theta_r\in\R^3$ (part of $\theta \in \R^{3\times24}$)

Global orient control the face direction of the virtual human, which is also the "rotation" of the root joint.

You may have to check [axis angle](https://en.wikipedia.org/wiki/Axis%E2%80%93angle_representation) before going on.
For example, a vector $\vec{r} = [x, y, z]$ represents a rotation around the axis $\frac{\vec{r}}{||\vec{r}||}$ in radians $||\vec{r}||$.

In [63]:
def learn_orient():
    fake_orient = torch.zeros(B, 1, 3)
    fake_orient[   : 50, :, 0] = torch.linspace(0, 2 * np.pi, 50).reshape(50, 1)  # about x-axis
    fake_orient[ 50:100, :, 1] = torch.linspace(0, 2 * np.pi, 50).reshape(50, 1)  # about y-axis
    fake_orient[100:150, :, :] = torch.linspace(0, 2 * np.pi, 50).reshape(50, 1, 1).repeat(1, 1, 3)  # about x=y=z

    # Inference.
    smpl_out = body_model(
            betas         = torch.zeros(B, 10),     # shape coefficients
            global_orient = fake_orient,            # axis-angle representation
            body_pose     = torch.zeros(B, 23, 3),  # axis-angle representation
            transl        = torch.zeros(B, 3),
        )

    # Check output.
    joints : torch.Tensor = smpl_out.joints    # (B, 45 3)
    verts  : torch.Tensor = smpl_out.vertices  # (B, 7890 3)

    def visualize_results():
        """ This part is to visualize the results. You are supposed to ignore this part. """
        shape_wis3d = Wis3D(
                pm.outputs / 'wis3d',
                'SMPL-parameters',
            )

        # Prepare the rotation axis.
        axis_x   = torch.tensor([[0, 0, 0], [3, 0, 0]], dtype=torch.float32)
        axis_y   = torch.tensor([[0, 0, 0], [0, 3, 0]], dtype=torch.float32)
        axis_xyz = torch.tensor([[0, 0, 0], [1, 1, 1]], dtype=torch.float32)
        axis_all = torch.concat(
            [
                axis_x.reshape(1, 2, 3).repeat(50, 1, 1),
                axis_y.reshape(1, 2, 3).repeat(50, 1, 1),
                axis_xyz.reshape(1, 2, 3).repeat(50, 1, 1),
            ],
            dim = 0,
        )
        axis_all[:, :, :] += joints[:, [0], :] # move the axis to the root joints


        shape_wis3d.add_vec_seq(
            vecs = axis_all,
            name = 'orient: rotation axis',
        )
        shape_wis3d.add_motion_verts(
                verts  = verts,
                name   = f'orient: vertices',
                offset = 0,
            )
        shape_wis3d.add_motion_skel(
                joints = joints[:, :24],
                bones  = Skeleton_SMPL24.bones,
                colors = Skeleton_SMPL24.bone_colors,
                name   = f'orient: skeleton',
                offset = 0,
            )
    visualize_results()

We will visualize the effects of the changes on `global_orient`.

Here, `learn_orient()` will rotate the digital human in three ways:

1. `fake_orient[  0: 50]` = $[0, 0, 0] \rightarrow [2\pi,  0, 0 ]$, rotation about $x$-axis
2. `fake_orient[ 50:100]` = $[0, 0, 0] \rightarrow [ 0, 2π, 0 ]$, rotation about $y$-axis
3. `fake_orient[100:150]` = $[0, 0, 0] \rightarrow [2π, 2π, 2π]$, rotation about $x=y=z$ axis

You are supposed to make sure you understand the axis-angle representation before going on.

In [ ]:
learn_orient()

Now start the Wis3D viewer.

You will see that, the rotation axis starts from the position of root joint, rather than the origin of the coordinates.

In [ ]:
!wis3d --vis_dir {pm.outputs / 'wis3d'} --host 0.0.0.0 --port 19090

There is still one thing you should know: the exact value of global orientation is related to the **coordinates** (e.g., camera coordinates, global coordinates) you are using. (So is the translation in SMPL.)

#### 3. body_pose | $\theta_b\in\R^{3\times23}$ (part of $\theta \in \R^{3\times24}$)

#### 4. transl | $\Gamma\in\R^{3}$